## EE232E Project 2
### Problem 1
Data preprocessing


In [1]:
from igraph import *
from collections import defaultdict
import string
import pickle
import re
from tqdm import tqdm as timer

In [2]:
def clean_string(s):
    return(re.sub(r'\(.*\)|\{.*\}|\'|\"', "", s).lstrip().rstrip())

In [6]:
infiles = ["./project_2_data/actor_movies.txt", "./project_2_data/actress_movies.txt"]
printable = set(string.printable)
fl = {"./project_2_data/actor_movies.txt": 2167653, "./project_2_data/actress_movies.txt": 1182813}

A2M = defaultdict(set)
M2A = defaultdict(set)

for f in infiles:
    with open(f, "rb") as infile:
        for line in timer(infile, total=fl[f], desc=f.split('/')[2]):
            if line.count("\t\t") <= 10:
                continue
            line = filter(lambda x : x in printable, line.strip().translate(None, "&$ "))
            arr = line.split("\t\t")
            
            actor = arr[0]
            movies = map(clean_string, arr[1:])
            
            for m in movies:
                A2M[actor].add(m)
                M2A[m].add(actor)


actress_movies.txt: 100%|██████████| 1182813/1182813 [00:06<00:00, 191425.44it/s]


In [7]:
print len(M2A)
print len(A2M)

393715
101124


In [8]:
# remove movies that have less than 10 actors in it.
large_movies = set(filter(lambda x : len(M2A[x]) > 10, M2A.keys()))
A2M = {actor : movies.intersection(large_movies) for (actor, movies) in A2M.iteritems()}
A2M = {actor : movies for (actor, movies) in A2M.iteritems() if len(movies) > 0}
M2A = {movie : actors for (movie, actors) in M2A.iteritems() if len(actors) > 10}

In [9]:
print len(A2M)
print len(M2A)

99652
90829


In [10]:
# uncomment this to save dictionary to local file
pickle.dump(A2M, open("A2M.pkl", "wb" ))
pickle.dump(M2A, open("M2A.pkl", "wb" ))

# uncomment this to load dictionary from local file
# A2M = pickle.load(open("A2M.pkl", "rb" ))
# M2A = pickle.load(open("M2A.pkl", "rb" ))

### Problem 2
Construct a weighted directed graph $G(V, E)$ from the list, while:

$V = \text{all actors/actresses in the list.} $

$S_i = \{m | i \in V, m \text{ is a movie in which } i \text{ has acted.} \}$

$ E = \{(i,j)|i,j ∈ V,S_i \cap S_j \neq ∅\} \text{ and for each directed Edge } i → j, \text{ a weight is assigned as } \frac{|S_i \cap S_j|}{|S_i|}.$

In [28]:
# create edge list
count = 0
outfile = open("actor_edgelist.txt", "w")

for i, movies in timer(A2M.iteritems(), total=len(A2M)):
    # for an actor i, get the list of actors that appear in the same movie
    i2k = defaultdict(int)
    for j in movies:
        for k in M2A[j]:
            i2k[k]=i2k[k]+1
    for k, w in i2k.iteritems():
        weight = float(w) / len(movies)
        line = i + '\t' + k + '\t' + str(weight) + '\n'
        outfile.write(line)
        count = count + 1
print(count)
outfile.close()

100%|██████████| 99652/99652 [00:59<00:00, 1677.70it/s]


42135556


In [29]:
g = Graph.Read_Ncol('actor_edgelist.txt', directed=True)
print(g.vcount())
print(g.ecount())

99652
42135556


### Problem 3
Run the Page Rank algorithm on the network. And list the top 10 actors according to page rank.
We also listed the top 10 actors ranked by the number of movie he/she has acted in.

In [11]:
page_rank = g.pagerank(vertices=None, directed=True)

In [12]:
sorted_pr = sorted(range(len(page_rank)), key=lambda k: page_rank[k], reverse=True)

In [13]:
for actor in g.vs[sorted_pr[0:10]]["name"]:
    print actor, len(A2M[actor]), page_rank[g.vs.find(actor).index]

Flowers,Bess 805 0.000169606118702
Harris,Sam(II) 597 0.000150271164822
Miller,Harold(I) 539 0.000137373359751
O'Brien,WilliamH. 429 0.000125948101775
Phelps,Lee(I) 626 0.000120533526418
Farnum,Franklyn 501 0.000119791816517
Sayre,Jeffrey 426 0.000118625691338
Holmes,Stuart(I) 436 0.000118068323525
Kemp,KennerG. 418 0.000116560842022
Steers,Larry 503 0.000112654999786


In [16]:
# sort by number of movies appeared in
sorted_A2M = sorted(A2M, key=lambda k: len(A2M[k]), reverse=True)
for actor in sorted_A2M[:10]:
    print actor, len(A2M[actor]), page_rank[g.vs.find(actor).index]

Flowers,Bess 805 0.000169606118702
Hack,Herman 657 6.79231399928e-05
Phelps,Lee(I) 626 0.000120533526418
O'Connor,Frank(I) 600 0.00010456007882
Harris,Sam(II) 597 0.000150271164822
Miller,Harold(I) 539 0.000137373359751
Ellis,Frank(I) 523 6.64753424485e-05
London,Tom 515 8.59256048855e-05
Steers,Larry 503 0.000112654999786
Farnum,Franklyn 501 0.000119791816517


### Problem 4
Construct a movie network according to the set of actors/actresses, with weight assigned as the jaccard index of the actor sets of 2 movies. Now we have an undirected network instead.

In [17]:
# helper function that computes jaccard index of two set
def jaccard_index(first_set, second_set):
    """ Computes jaccard index of two sets
        Arguments:
          first_set(set):
          second_set(set):
        Returns:
          index(float): Jaccard index between two sets; it is 
            between 0.0 and 1.0
    """
    # If both sets are empty, jaccard index is defined to be 1
    index = 1.0
    if first_set or second_set:
        index = (float(len(first_set.intersection(second_set))) / len(first_set.union(second_set)))

    return index


In [32]:
# create movie edge list for movie with at least 10 actors/actresses
count = 0
outfile = open("movie_edgelist.txt", "w")

processed_movie = set()
for i, actors in timer(M2A.iteritems(), total=len(M2A)):
    # for a movie i, get the list of movies that share some actors
    if i == "":
        continue
    unique_movies = set()
    for j in actors:
        for k in A2M[j]:
            if k in processed_movie:
                continue
            weight = jaccard_index(actors, M2A[k])
            unique_movies.add( (k, weight) )
    for (k, weight) in unique_movies:
        line = i + '\t' + k + '\t' + str(weight) + '\n'
        outfile.write(line)
        count = count + 1
    processed_movie.add(i)
#     for k, w in i2k.iteritems():
#         weight = jaccard_index()
#         line = i + '\t' + k + '\t' + str(weight) + '\n'
#         outfile.write(line)
#         count = count + 1
outfile.close()
print(count)

100%|██████████| 90829/90829 [04:48<00:00, 315.15it/s] 

41030666


In [3]:
# construct the movie graph
movie_g = Graph.Read_Ncol('movie_edgelist.txt', directed=False)

In [11]:
# a dictionary that maps movie to genre
M2G = {}
printable = set(string.printable)

with open('./project_2_data/movie_genre.txt', 'rb') as infile:
    for line in timer(infile, total=1010991, desc="movie_genre.txt"):
        line = filter(lambda x : x in printable, line.strip().translate(None, "&$ "))
        arr = line.split('\t\t')
        movie = clean_string(arr[0])
        genre = clean_string(arr[1])
        
        if movie in M2A:
            M2G[movie] = genre

movie_genre.txt: 100%|██████████| 1010991/1010991 [00:07<00:00, 134983.64it/s]


In [ ]:
result = movie_g.community_fastgreedy()

In [44]:
M2G


0.0344827586207
0.0232558139535
0.01
0.0294117647059
0.0384615384615
0.0416666666667
0.046511627907
0.0333333333333
0.0344827586207
0.0142857142857


In [ ]:
movie_g.vs.find("Minions2015")